# Charger le modèle depuis le fichier .h5

In [ ]:
model = # A COMPLETER
print("Modèle chargé avec succès.")

# Vérifier la structure du modèle pour identifier la dernière couche de convolution

In [ ]:
model.summary()

# Générer la carte de chaleur Grad-CAM

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

# Fonction pour générer la carte de chaleur Grad-CAM
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    # Obtenir le modèle pour la carte de chaleur Grad-CAM
    grad_model = Model(
        [model.inputs], 
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Calculer le gradient de la classe prédite par rapport à la dernière couche de convolution
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        predicted_class = tf.argmax(predictions[0])
        loss = predictions[:, predicted_class]

    grads = tape.gradient(loss, conv_outputs)

    # Calculer la moyenne des gradients le long de l'axe de profondeur (canal)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiplier chaque canal par son poids moyen
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Appliquer ReLU pour ne conserver que les valeurs positives
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
# Fonction pour superposer la carte de chaleur sur l'image originale
def superimpose_heatmap(img, heatmap, alpha=0.4):
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    return superimposed_img

# Test

In [ ]:
# Charger une image de test
image_path = random.choice(random_images)
img = tf.keras.utils.load_img(image_path, target_size=(256, 256))
img_array = tf.keras.utils.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Générer la carte de chaleur Grad-CAM
last_conv_layer_name = "conv2d_2"  # Dernière couche de convolution de ton modèle
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Convertir l'image en format BGR pour OpenCV
img_bgr = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

# Superposer la carte de chaleur sur l'image originale
superimposed_img = superimpose_heatmap(img_bgr, heatmap)

# Afficher l'image originale, la carte de chaleur et la superposition
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.title("Image d'origine")
plt.imshow(np.array(img))
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Carte de chaleur Grad-CAM")
plt.imshow(heatmap, cmap="jet")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Superposition")
plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
plt.axis("off")

plt.tight_layout()
plt.show()